In [1]:
import pandas as pd 
import numpy as np
import openpyxl
import os

In [4]:
df= pd.read_excel('TG101_INVENTORY.xlsx')
df.head()

,PART,CURRENT_USAGE,PRIOR_YEAR_USAGE,TOTAL_ON_HAND,PLANNED_REQ,PROD_REQUIRED,SAFETY_STOCK,PLANNED_ORDERS,COST,YTD_WEEKLY_USAGE,FORECASTED_YEARLY_USAGE,Frequency,Vendor Number,Lead_Time,STD_DEV_LEAD_TIME,Order Qty
0,TG016L122,538.00,1400.0,49.0,35.0,26.0,250,250,3.0310,20.69,1076.00,6,BIO001,19.0,20.011719,250.0
1,TG016L146,959.01,2079.0,-156.0,34.0,64.0,250,500,3.6271,36.89,1918.02,6,BIO001,19.0,20.032242,250.0
2,TG016L170,1109.00,3011.0,-20.0,136.0,68.0,250,250,4.2231,42.65,2218.00,6,BIO001,19.0,20.052825,250.0
3,TG016L194,1065.00,2512.0,97.0,34.0,71.0,250,250,4.8192,40.96,2130.00,6,BIO001,19.0,20.073466,250.0
4,TG016L242,1004.00,2372.0,64.0,67.0,81.0,300,300,6.0113,38.62,2008.00,7,BIO001,19.0,20.094167,300.0


In [5]:
df['Lead_Time'] = df['Lead_Time'].fillna(25)
df['STD_DEV_LEAD_TIME'] = df['STD_DEV_LEAD_TIME'].fillna(19)
df['Frequency'] = df['Frequency'].fillna(0)
df['Order Qty'] = df['Order Qty'].fillna(0)
df['Daily_Usage'] = df['YTD_WEEKLY_USAGE'] / 7

In [6]:
df['YTD_WEEKLY_USAGE'] 

0     20.69
1     36.89
2     42.65
3     40.96
4     38.62
      ...  
66    45.27
67    14.19
68    27.12
69     0.19
70     0.08
Name: YTD_WEEKLY_USAGE, Length: 71, dtype: float64

## Function to Calculate Weighted Annual Demand

In [7]:
def calculate_annual_demand(row):
    current = row['CURRENT_USAGE'] or 0
    prior = row['PRIOR_YEAR_USAGE'] or 0
    forecast = row['FORECASTED_YEARLY_USAGE'] or 0
    frequency = row['Frequency'] or 0
    order_qty = row['Order Qty'] or 0
    
    if current > 0 and prior > 0:
        # High confidence: trend-adjusted weighted average
        trend = max(min(current / prior, 1.5), 0.5)  # Cap extreme trends
        demand = (current * 0.7) + (prior * trend * 0.3)
        confidence = 'HIGH'
    elif current > 0:
        # Medium confidence: current with growth
        demand = current * 1.05
        confidence = 'MEDIUM'
    elif frequency > 0 and order_qty > 0:
        # Low confidence: frequency-based
        demand = frequency * order_qty
        confidence = 'LOW'
    else:
        # Very low confidence: use forecast
        demand = forecast
        confidence = 'VERY_LOW'
    
    return demand, confidence

## Function to Calculate Safety Stock


In [8]:
def calculate_safety_stock(row, service_level=0.95):
    daily_usage = row['Daily_Usage']
    lead_time = row['Lead_Time']
    lead_time_std = row['STD_DEV_LEAD_TIME']
    current = row['CURRENT_USAGE'] or 0
    prior = row['PRIOR_YEAR_USAGE'] or 0
    
    # Service level z-score (95% = 1.65, 99% = 2.33)
    z_score = 1.65 if service_level == 0.95 else 2.33
    
    # Calculate demand coefficient of variation
    if current > 0 and prior > 0:
        demand_cv = abs(current - prior) / ((current + prior) / 2)
    else:
        demand_cv = 0.3  # Default 30% CV
    
    # Lead time variability buffer
    lt_variability = z_score * daily_usage * lead_time_std
    
    # Demand variability buffer
    demand_variability = z_score * daily_usage * demand_cv * np.sqrt(lead_time)
    
    # Total safety stock (minimum of calculated or current safety stock)
    calculated_safety = lt_variability + demand_variability
    current_safety = row['SAFETY_STOCK'] or 0
    
    return max(calculated_safety, current_safety)

## Function to Calculate EOQ

In [9]:
def calculate_eoq(annual_demand, unit_cost, order_cost=50, holding_rate=0.25):
    if annual_demand > 0 and unit_cost > 0:
        eoq = np.sqrt((2 * annual_demand * order_cost) / (unit_cost * holding_rate))
        return eoq
    return 0


## Bring everything to the BBQ

In [ ]:
results = []

for index, row in df.iterrows():
    # Calculate annual demand
    annual_demand, confidence = calculate_annual_demand(row)
    
    # Calculate lead time demand
    lead_time_demand = row['Daily_Usage'] * row['Lead_Time']
    
    # Calculate safety stock
    safety_stock = calculate_safety_stock(row)
    
    # Calculate reorder point
    reorder_point = lead_time_demand + safety_stock
    
    # Calculate EOQ
    eoq = calculate_eoq(annual_demand, row['COST'])
    if eoq == 0 and row['Order Qty'] > 0:
        eoq = row['Order Qty']  # Use existing if EOQ calc fails
    
    # Determine action urgency
    current_stock = row['TOTAL_ON_HAND'] or 0
    days_of_stock = row['Daily_Usage'] > 0 and current_stock / row['Daily_Usage'] or 999
    
    if current_stock < 0:
        urgency = 'CRITICAL'
        message = 'STOCKOUT - Expedite order'
    elif current_stock < lead_time_demand:
        urgency = 'URGENT'
        message = 'Below lead time demand'
    elif current_stock < reorder_point:
        urgency = 'SOON'
        message = 'Below reorder point'
    elif days_of_stock < row['Lead_Time'] * 1.5:
        urgency = 'MONITOR'
        message = 'Approaching reorder point'
    else:
        urgency = 'OK'
        message = 'Adequate inventory'
    
    # Calculate order value
    order_value = eoq * row['COST']
    
    # Store results
    results.append({
        'PART': row['PART'],
        'Vendor': row['Vendor Number'],
        'Annual_Demand': round(annual_demand, 2),
        'Confidence': confidence,
        'Lead_Time_Demand': round(lead_time_demand, 2),
        'Safety_Stock': round(safety_stock, 2),
        'Reorder_Point': round(reorder_point, 2),
        'EOQ': round(eoq, 0),
        'Current_Stock': current_stock,
        'Days_of_Stock': round(days_of_stock, 1),
        'Action_Urgency': urgency,
        'Action_Message': message,
        'Order_Value': round(order_value, 2),
        'Annual_Value': round(annual_demand * row['COST'], 2)
    })

# Create results DataFrame
results_df = pd.DataFrame(results)

# Summary statistics
print("=== PROACTIVE ORDERING ANALYSIS SUMMARY ===\n")

urgency_summary = results_df.groupby('Action_Urgency').agg({
    'PART': 'count',
    'Order_Value': 'sum'
}).round(2)

print("Parts by Urgency:")
print(urgency_summary)

# Immediate ordering needs
immediate_orders = results_df[results_df['Action_Urgency'].isin(['CRITICAL', 'URGENT'])]
total_immediate_value = immediate_orders['Order_Value'].sum()

print(f"\nImmediate ordering required:")
print(f"Parts: {len(immediate_orders)}")
print(f"Total value: ${total_immediate_value:,.2f}")

# Top priorities
print(f"\nTop 10 ordering priorities:")
top_priorities = results_df[results_df['Action_Urgency'].isin(['CRITICAL', 'URGENT', 'SOON'])].nlargest(10, 'Order_Value')

for i, (_, row) in enumerate(top_priorities.iterrows(), 1):
    print(f"{i:2d}. {str(row['PART']):12s} {str(row['Action_Urgency']):8s} Order {row['EOQ']:6.0f} units (${row['Order_Value']:8,.0f})")

# Write results back to Excel
# This will create a new sheet with the analysis results
try:
    with pd.ExcelWriter('TG101_INVENTORY_ANALYSIS.xlsx', engine='openpyxl') as writer:
        results_df.to_excel(writer, sheet_name='ProactiveOrderingAnalysis', index=False)
    print(f"\nResults written to 'TG101_INVENTORY_ANALYSIS.xlsx' file")
except Exception as e:
    print(f"\nError writing to Excel: {e}")
    print("Displaying results summary instead:")
    print(results_df.head(10))

print(f"\nTotal parts analyzed: {len(results_df)}")

=== PROACTIVE ORDERING ANALYSIS SUMMARY ===

Parts by Urgency:
                PART  Order_Value
Action_Urgency                   
CRITICAL          15     22518.27
OK                19     43078.66
SOON              21     63299.60
URGENT            16     36745.24

Immediate ordering required:
Parts: 31
Total value: $59,263.51

Top 10 ordering priorities:
 1. URAISOLB     SOON     Order   9048 units ($  16,558)
 2. 2004         SOON     Order  32583 units ($   7,175)
 3. TG2006       URGENT   Order   6892 units ($   4,941)
 4. UTGDTAPE     SOON     Order  77761 units ($   3,966)
 5. TG084L192    URGENT   Order    740 units ($   3,377)
 6. TG084L120    CRITICAL Order   1177 units ($   3,355)
 7. TG084L170    SOON     Order    804 units ($   3,245)
 8. TG084L108    URGENT   Order   1233 units ($   3,164)
 9. 2008TG1000WA URGENT   Order    270 units ($   2,938)
10. 2008TG1600WA URGENT   Order    170 units ($   2,936)

Results written to 'TG101_INVENTORY_ANALYSIS.xlsx' file

Total parts 

: 